# Neighborhood Analysis in Toronto for Traveller

# Introduction

As a travel lover, I've been to a lot of Countries and cities. Unlike a lot of tourists, I love to talk to native, live in a different place, and enjoy another kind of lifestyle. So when I choose where I should live when traveling, I do not care much about is there any tourists' attractions nearby. What I care is how can I enjoy myself and live like a native. 

Toronto, the biggest city in Canada, is always on my list. I not only want to travel there but also want to live there. 

To realize my "daydream," I need to do some research about this city to decide which area is the best place for me to live. So the biggest problem of this project is: Where should I live in Toronto? There are few criterion when I choose the Neighborhood.

    I hope this Neighborhood has some Chinese restaurants, so that I can satisfy my Chinese appetite.

    I also want this place near the pharmacy, since I'm not in good health and need to buy some medicine all the time. 

    And this place needs to have some supermarket or grocery store, where I can buy some food and cook at home. 

    Last but not least, I hope this Neighborhood has a nice bar. It's always good to spend time with friends or even strangers with beer. 

# Data Source and Description 

1. Neighborhood data of Toronto comes from Wikipedia. It contains the postcode number, neighborhood and borough data. However, it's not corresponding; one postcode may include two neighborhood. Therefore we have to aggregate the dataset after we parse the website page. 

2. Neighborhood latitude and longitude dataset. This dataset is offered by Coursera. Because Google Maps Geocoding API is not free anymore and Geocoder Python package is sometimes unreliable, especially for the postcode dataset like this, which is not corresponding, I choose to use the dataset, which contains all the latitude and longitude information of Toronto. 

3. Foursquare API. After we get the latitude, longitude and neighborhood dataset, we can make a call to Foursquare and get the venues data. Foursquare API is free for the limited number of requests, and I think it's enough for this project. 

# Technical Report

# Parsing from Wikipedia


Import all package at first. 

In [1]:
import pandas as pd
import numpy as np
import urllib.request, urllib.parse, urllib.error
from bs4 import BeautifulSoup
import requests
import ssl
import re
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

import json # library to handle JSON files

from sklearn.cluster import KMeans
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
!pip install folium

import folium

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

Requirement not upgraded as not directly required: folium in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages
Requirement not upgraded as not directly required: branca>=0.3.0 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: jinja2 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: numpy in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: requests in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: six in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: MarkupSafe>=0.23 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from jinja2->folium)
Requirement not upgraded as not directly required: chardet<3.1.0,>=

Use the BeautifulSoup package to parse the data from Wikipedia and check what we have. 

In [2]:
url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

soup = BeautifulSoup(url, 'lxml')
x = soup.find('table', {'class':'wikitable sortable'})
print(x)

<table class="wikitable sortable">
<tbody><tr>
<th>Postcode</th>
<th>Borough</th>
<th>Neighbourhood
</th></tr>
<tr>
<td>M1A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M2A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
</td></tr>
<tr>
<td>M4A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Harbourfront_(Toronto)" title="Harbourfront (Toronto)">Harbourfront</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Regent_Park" title="Regent Park">Regent Park</a>
</td></tr>
<tr>
<td>M6A</td>

Looks good! It's clear and contain all the infomation we need to know, postcode, borough and neighborhood. 
Let's put all these data into frame, we need to use pandas package here. 

In [3]:
column_names = ['Postcode','Borough','Neighborhood']
tor = pd.DataFrame(columns = column_names)
postcode = 0
borough = 0
neighborhood = 0

for tr in x.find_all('tr'):
    i = 0
    for td in tr.find_all('td'):
        if i == 0:
            postcode = td.text
            i = i + 1
        elif i == 1:
            borough = td.text
            i = i + 1
        elif i == 2: 
            neighborhood = td.text[0:-1]
    tor = tor.append({'Postcode': postcode,'Borough': borough,'Neighborhood': neighborhood},ignore_index=True)
    print(postcode, borough, neighborhood)

0 0 0
M1A Not assigned Not assigned
M2A Not assigned Not assigned
M3A North York Parkwoods
M4A North York Victoria Village
M5A Downtown Toronto Harbourfront
M5A Downtown Toronto Regent Park
M6A North York Lawrence Heights
M6A North York Lawrence Manor
M7A Queen's Park Not assigned
M8A Not assigned Not assigned
M9A Etobicoke Islington Avenue
M1B Scarborough Rouge
M1B Scarborough Malvern
M2B Not assigned Not assigned
M3B North York Don Mills North
M4B East York Woodbine Gardens
M4B East York Parkview Hill
M5B Downtown Toronto Ryerson
M5B Downtown Toronto Garden District
M6B North York Glencairn
M7B Not assigned Not assigned
M8B Not assigned Not assigned
M9B Etobicoke Cloverdale
M9B Etobicoke Islington
M9B Etobicoke Martin Grove
M9B Etobicoke Princess Gardens
M9B Etobicoke West Deane Park
M1C Scarborough Highland Creek
M1C Scarborough Rouge Hill
M1C Scarborough Port Union
M2C Not assigned Not assigned
M3C North York Flemingdon Park
M3C North York Don Mills South
M4C East York Woodbine Hei

M3Z Not assigned Not assigned
M4Z Not assigned Not assigned
M5Z Not assigned Not assigned
M6Z Not assigned Not assigned
M7Z Not assigned Not assigned
M8Z Etobicoke Kingsway Park South West
M8Z Etobicoke Mimico NW
M8Z Etobicoke The Queensway West
M8Z Etobicoke Royal York South West
M8Z Etobicoke South of Bloor
M9Z Not assigned Not assigned


In [4]:
tor = tor[tor.Borough!='Not assigned']
tor = tor[tor.Neighborhood!='Not assigned']
tor = tor[tor.Borough!= 0]
tor.reset_index(drop = True, inplace = True)

tor = tor.groupby(['Postcode','Borough'])['Neighborhood'].apply(', '.join).reset_index()

tor.head()

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Like I said before, a single postcode may contain many neiborhood areas. Therefore I have to aggregate the whole dataset, and it shows all neiborhoods in one postcode area. This can help us have a clear idea about the area, and make the job easier in the next step. 

Next, we use the dataset we downloaded from Coursera website, and aggregate the two dataset together.

In [5]:
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share your notebook.
client_dff8a113d75e412297d555c23139b4ae = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='A1hpE2gUdK0I7nXY9z-EkAGvnh7zYopG7cZcYvRI-mxl',
    ibm_auth_endpoint="https://iam.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_dff8a113d75e412297d555c23139b4ae.get_object(Bucket='myjupyternotebookonibmdatascience-donotdelete-pr-sdqy5hblun8tew',Key='Geospatial_Coordinates.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_1 = pd.read_csv(body)
df_data_1.rename(columns={'Postal Code':'Postcode'}, inplace=True)

df_data_1.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [6]:
tor.set_index('Postcode')
df_data_1.set_index('Postcode')
toronto_data = pd.merge(tor, df_data_1)
toronto_data

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


Looks great! Let's take a look where are these place on the map! Always good to visualize the number. 

In [7]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [8]:
map_tor = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_tor)  
    
map_tor

Now we have to use Foursquare API to get the venues data around these neighborhood. 
After we get the url, let's run a test, randomly choose a number and try to get the venues information. 

# Get Venues data from Foursquare

In [9]:
CLIENT_ID = 'KFCMPSJJJKYAMYNH0DTGZ3AMLNOIQ5S5E0ZN1QI2J5PZNLNT' # your Foursquare ID
CLIENT_SECRET = 'CZ2KFRPJSMDIDXLM5EAXENRTKR33SVG2ZPCJTZQZVCD5DMRJ' # your Foursquare Secret
VERSION = '20190501' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: KFCMPSJJJKYAMYNH0DTGZ3AMLNOIQ5S5E0ZN1QI2J5PZNLNT
CLIENT_SECRET:CZ2KFRPJSMDIDXLM5EAXENRTKR33SVG2ZPCJTZQZVCD5DMRJ


In [10]:
toronto_data.loc[67, 'Neighborhood']

'Chinatown, Grange Park, Kensington Market'

In [11]:
neighborhood_latitude = toronto_data.loc[67, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_data.loc[67, 'Longitude'] # neighborhood longitude value

neighborhood_name = toronto_data.loc[67, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Chinatown, Grange Park, Kensington Market are 43.6532057, -79.4000493.


In [12]:
# set the radius and how many venues you want to get for each neighborhood
radius = 500
LIMIT = 100
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=KFCMPSJJJKYAMYNH0DTGZ3AMLNOIQ5S5E0ZN1QI2J5PZNLNT&client_secret=CZ2KFRPJSMDIDXLM5EAXENRTKR33SVG2ZPCJTZQZVCD5DMRJ&v=20190501&ll=43.6532057,-79.4000493&radius=500&limit=100'

In [13]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ccfae70351e3d1d525befd9'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4b116957f964a520087c23e3-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/artstore_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d127951735',
         'name': 'Arts & Crafts Store',
         'pluralName': 'Arts & Crafts Stores',
         'primary': True,
         'shortName': 'Arts & Crafts'}],
       'id': '4b116957f964a520087c23e3',
       'location': {'address': '205 Augusta Ave.',
        'cc': 'CA',
        'city': 'Toronto',
        'country': 'Canada',
        'crossStreet': 'Denison Square',
        'distance': 156,
        'formattedAddress': ['205 Augusta Ave. (Denison Square)',
         'Toronto ON M5T 2L4',
         'Canada'],


Great! We have the json file now. Among all these information about venues, the most important one for this project is categories, which can help me determine where I want to live. Let's locate the categories from all these variables and put them into a dataframe. 

In [14]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [15]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

,name,categories,lat,lng
0,Kid Icarus,Arts & Crafts Store,43.653933,-79.401719
1,Seven Lives - Tacos y Mariscos,Mexican Restaurant,43.654418,-79.400545
2,Essence of Life Organics,Organic Grocery,43.654111,-79.400431
3,Little Pebbles,Coffee Shop,43.654883,-79.400264
4,Jimmy's Coffee,Café,43.654493,-79.401311
5,The Moonbean Cafe,Café,43.654147,-79.400182
6,Blackbird Baking Co,Bakery,43.654764,-79.400566
7,El Rey,Cocktail Bar,43.652764,-79.400048
8,FIKA Cafe,Café,43.653560,-79.400402
9,Banh Mi Nguyen Huong,Vietnamese Restaurant,43.653628,-79.398376


In [16]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


Our test is running good. Next we have to apply this method on all neighborhoods. 

In [17]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [19]:
tor_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
Silver Hills, York Mills
Newtonbrook, Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park, Don Mills South
Bathurst Manor, Downsview North, Wilson Heights
Northwood Park, York University
CFB Toronto, Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens, Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West, 

In [25]:
k = tor_venues
print(tor_venues.shape)
tor_venues.head()

(2205, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
3,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Marina Spa,43.766000,-79.191000,Spa


There are about two thousand venues record here, it's better to group the data by neighborhood. Therefore we can understand how many venues in a single neighborhood. 

In [26]:
tor_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Agincourt,4,4,4,4,4,4
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",3,3,3,3,3,3
"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",9,9,9,9,9,9
"Alderwood, Long Branch",10,10,10,10,10,10
"Bathurst Manor, Downsview North, Wilson Heights",19,19,19,19,19,19
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",23,23,23,23,23,23
Berczy Park,57,57,57,57,57,57


In [27]:
print('There are {} uniques categories.'.format(len(tor_venues['Venue Category'].unique())))

There are 272 uniques categories.


Now the table above shows us how many venues in a neighborhood, however ithe table is too big and contains many place I don't interested in. For example, the last one York Mills West only has 3 venues. It means that this place do not have many venues, or the data set do not have the data in this neighborhood. I want to live in some popular area, not this kind of "middle of nowhere"

Therefore, next I should focus on the area which has many venues, and contains the venues I interested in, such as "chinese food restaurant", "Grocery Store."

In [28]:
# one hot encoding
tor_onehot = pd.get_dummies(tor_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
tor_onehot['Neighborhood'] = tor_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [tor_onehot.columns[-1]] + list(tor_onehot.columns[:-1])
tor_onehot = tor_onehot[fixed_columns]

tor_onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Housing Development,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Rock Climbing Spot,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [29]:
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
tor_grouped = tor_onehot.groupby('Neighborhood').mean().reset_index()
tor_grouped

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Housing Development,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Moving Target,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Rock Climbing Spot,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,"Adelaide, King, Richmond",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.

Check all kinds of the venues. And choose the ones I interested in. 

In [30]:
print(tor_grouped.columns.values)

['Neighborhood' 'Yoga Studio' 'Accessories Store' 'Adult Boutique'
 'Afghan Restaurant' 'Airport' 'Airport Food Court' 'Airport Gate'
 'Airport Lounge' 'Airport Service' 'Airport Terminal'
 'American Restaurant' 'Antique Shop' 'Aquarium' 'Arcade'
 'Argentinian Restaurant' 'Art Gallery' 'Art Museum' 'Arts & Crafts Store'
 'Asian Restaurant' 'Athletics & Sports' 'Auto Garage' 'Auto Workshop'
 'BBQ Joint' 'Baby Store' 'Bagel Shop' 'Bakery' 'Bank' 'Bar'
 'Baseball Field' 'Baseball Stadium' 'Basketball Stadium' 'Beach'
 'Bed & Breakfast' 'Beer Bar' 'Beer Store' 'Belgian Restaurant' 'Bike Shop'
 'Bistro' 'Boat or Ferry' 'Bookstore' 'Boutique' 'Brazilian Restaurant'
 'Breakfast Spot' 'Brewery' 'Bridal Shop' 'Bubble Tea Shop' 'Burger Joint'
 'Burrito Place' 'Bus Line' 'Bus Station' 'Business Service' 'Butcher'
 'Cafeteria' 'Café' 'Cajun / Creole Restaurant' 'Camera Store'
 'Candy Store' 'Caribbean Restaurant' 'Cheese Shop' 'Chinese Restaurant'
 'Chocolate Shop' 'Church' 'Climbing Gym' 'Clothin

In [31]:
new = tor_grouped[['Neighborhood','Grocery Store','Chinese Restaurant','Dim Sum Restaurant','Bar','Pharmacy','Hospital']]
new

,Neighborhood,Grocery Store,Chinese Restaurant,Dim Sum Restaurant,Bar,Pharmacy,Hospital
0,"Adelaide, King, Richmond",0.000000,0.000000,0.000000,0.030000,0.000000,0.00
1,Agincourt,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
2,"Agincourt North, L'Amoreaux East, Milliken, St...",0.000000,0.000000,0.000000,0.000000,0.000000,0.00
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",0.222222,0.000000,0.000000,0.000000,0.111111,0.00
4,"Alderwood, Long Branch",0.000000,0.000000,0.000000,0.000000,0.100000,0.00
5,"Bathurst Manor, Downsview North, Wilson Heights",0.000000,0.000000,0.000000,0.000000,0.052632,0.00
6,Bayview Village,0.000000,0.250000,0.000000,0.000000,0.000000,0.00
7,"Bedford Park, Lawrence Manor East",0.043478,0.000000,0.000000,0.000000,0.043478,0.00
8,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
9,"Birch Cliff, Cliffside West",0.000000,0.000000,0.000000,0.000000,0.000000,0.00


In [32]:
new['Total'] = new['Grocery Store']+new['Chinese Restaurant']+new['Dim Sum Restaurant']+new['Bar']+new['Pharmacy']+new['Hospital']
new

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


,Neighborhood,Grocery Store,Chinese Restaurant,Dim Sum Restaurant,Bar,Pharmacy,Hospital,Total
0,"Adelaide, King, Richmond",0.000000,0.000000,0.000000,0.030000,0.000000,0.00,0.030000
1,Agincourt,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
2,"Agincourt North, L'Amoreaux East, Milliken, St...",0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",0.222222,0.000000,0.000000,0.000000,0.111111,0.00,0.333333
4,"Alderwood, Long Branch",0.000000,0.000000,0.000000,0.000000,0.100000,0.00,0.100000
5,"Bathurst Manor, Downsview North, Wilson Heights",0.000000,0.000000,0.000000,0.000000,0.052632,0.00,0.052632
6,Bayview Village,0.000000,0.250000,0.000000,0.000000,0.000000,0.00,0.250000
7,"Bedford Park, Lawrence Manor East",0.043478,0.000000,0.000000,0.000000,0.043478,0.00,0.086957
8,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
9,"Birch Cliff, Cliffside West",0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000


In [33]:
new = new[(new['Total'] != 0)]
new = new[new['Grocery Store'] != 0]
new = new[new['Pharmacy'] != 0]

new

,Neighborhood,Grocery Store,Chinese Restaurant,Dim Sum Restaurant,Bar,Pharmacy,Hospital,Total
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",0.222222,0.000000,0.00,0.00,0.111111,0.00,0.333333
7,"Bedford Park, Lawrence Manor East",0.043478,0.000000,0.00,0.00,0.043478,0.00,0.086957
20,"Chinatown, Grange Park, Kensington Market",0.020000,0.030000,0.02,0.05,0.010000,0.01,0.140000
61,L'Amoreaux West,0.153846,0.153846,0.00,0.00,0.076923,0.00,0.384615
89,Thorncliffe Park,0.058824,0.000000,0.00,0.00,0.058824,0.00,0.117647
93,Willowdale South,0.029412,0.000000,0.00,0.00,0.029412,0.00,0.058824
94,Willowdale West,0.200000,0.000000,0.00,0.00,0.200000,0.00,0.400000


I delete the neighborhood data, which do not have Grocery store and pharmacy, because these two are the most important ones. 
And we get 7 neighborhoods now for us to choose. 

We can list the top 10 Venues of each neighborhood, in order to get a brief idea about what kind of shops is the most popular in each neighborhood. 

In [34]:
num_top_venues = 5

for hood in new['Neighborhood']:
    print("----"+hood+"----")
    temp = new[new['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 3})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown----
                venue   freq
0               Total  0.333
1       Grocery Store  0.222
2            Pharmacy  0.111
3  Chinese Restaurant  0.000
4  Dim Sum Restaurant  0.000


----Bedford Park, Lawrence Manor East----
                venue   freq
0               Total  0.087
1       Grocery Store  0.043
2            Pharmacy  0.043
3  Chinese Restaurant  0.000
4  Dim Sum Restaurant  0.000


----Chinatown, Grange Park, Kensington Market----
                venue  freq
0               Total  0.14
1                 Bar  0.05
2  Chinese Restaurant  0.03
3       Grocery Store  0.02
4  Dim Sum Restaurant  0.02


----L'Amoreaux West----
                venue   freq
0               Total  0.385
1       Grocery Store  0.154
2  Chinese Restaurant  0.154
3            Pharmacy  0.077
4  Dim Sum Restaurant  0.000


----Thorncliffe Park----
                venue   freq
0             

Let's visualize these 7 places and see where are they on the map. 

In [35]:
new.set_index('Neighborhood')
toronto_data.set_index('Neighborhood')
vis = pd.merge(new, toronto_data)
vis

,Neighborhood,Grocery Store,Chinese Restaurant,Dim Sum Restaurant,Bar,Pharmacy,Hospital,Total,Postcode,Borough,Latitude,Longitude
0,"Albion Gardens, Beaumond Heights, Humbergate, ...",0.222222,0.000000,0.00,0.00,0.111111,0.00,0.333333,M9V,Etobicoke,43.739416,-79.588437
1,"Bedford Park, Lawrence Manor East",0.043478,0.000000,0.00,0.00,0.043478,0.00,0.086957,M5M,North York,43.733283,-79.419750
2,"Chinatown, Grange Park, Kensington Market",0.020000,0.030000,0.02,0.05,0.010000,0.01,0.140000,M5T,Downtown Toronto,43.653206,-79.400049
3,L'Amoreaux West,0.153846,0.153846,0.00,0.00,0.076923,0.00,0.384615,M1W,Scarborough,43.799525,-79.318389
4,Thorncliffe Park,0.058824,0.000000,0.00,0.00,0.058824,0.00,0.117647,M4H,East York,43.705369,-79.349372
5,Willowdale South,0.029412,0.000000,0.00,0.00,0.029412,0.00,0.058824,M2N,North York,43.770120,-79.408493
6,Willowdale West,0.200000,0.000000,0.00,0.00,0.200000,0.00,0.400000,M2R,North York,43.782736,-79.442259


In [36]:
map_new = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(vis['Latitude'], vis['Longitude'], vis['Borough'], vis['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_new)  
    
map_new